# FLAML 을 이용한 AutoML 분석
## 사기탐지 신용카드 데이터분석

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 신원 도용, 계정 인수, 거래 사기를 포함하되 이에 국한되지 않는 다양한 유형의 사기에 대한 데이터를 수집하는 것이 포함됨

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

# 밑에 실행하면서 뜨는 빨간 경고 팝업 뜨지 않게 하기

In [3]:
!pip install autogluon

In [4]:
!pip install chart-studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.3 MB/s eta 0:00:00


In [5]:
!pip install pmdarima


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00


In [6]:
!pip install koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.1 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib   # 한국어 출력되게
import os
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
import plotly.graph_objs as go
import pmdarima as pm
from pmdarima.arima import ndiffs
from autogluon.tabular import TabularPredictor

### 칼럼 설명

- trans_date_trans_time (object): 거래가 발생한 날짜 및 시간
- cc_num (int64): 사용된 신용카드 번호를 나타내는 숫자
- merchant (object): 상점명 또는 거래가 발생한 가맹점의 이름
- category (object): 거래가 발생한 상품 또는 서비스의 범주
- amt (float64): 해당 거래의 거래 금액
- first (object): 카드 소유자의 이름
- last (object): 카드 소유자의 성
- gender (object): 카드 소유자의 성별
- street (object): 카드 소유자의 거주지 주소의 거리
- city (object): 카드 소유자의 거주지 도시
- state (object): 카드 소유자의 거주지 주(State)
- zip (int64): 카드 소유자의 거주지 우편번호
- lat (float64): 카드 소유자의 거주지 위도(latitude)
- long (float64): 카드 소유자의 거주지 경도(longitude)
- city_pop (int64): 카드 소유자가 거주하는 도시의 인구 수
- job (object): 카드 소유자의 직업
- dob (object): 카드 소유자의 출생일(Date of Birth)
- trans_num (object): 각 거래에 할당된 고유 거래 번호
- unix_time (int64): 거래가 발생한 시간을 유닉스 타임스탬프 형식으로 나타낸 값
- merch_lat (float64): 가맹점의 위도(latitude) 거래가 발생한 상점의 위치 정보
- merch_long (float64): 가맹점의 경도(longitude)
- is_fraud (int64): 거래가 사기(fraud)여부를 나타내는 열입니다. 1은 사기, 0은 정상 거래


In [9]:
data = pd.read_csv("/content/drive/MyDrive/creaditcard_fraud_data.csv")

data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [ ]:
train_data = data.dropna()
target_column = 'is_fraud'

model = TabularPredictor(label=target_column).fit(train_data)
test_data = train_data.sample(frac=0.2, random_state=10)
pred = model.predict(test_data)

print("Train Score:", model.evaluate(test_data))
print("best model: ", model.get_model_best())


No path specified. Models will be saved in: "AutogluonModels/ag-20241016_052005"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.14 GB / 12.67 GB (80.0%)
Disk Space Avail:   186.87 GB / 225.83 GB (82.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

[1000]	valid_set's binary_error: 0.00156554
[2000]	valid_set's binary_error: 0.00113366


	0.9989	 = Validation score   (accuracy)
	1075.43s	 = Training   runtime
	6.74s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.00161952
[2000]	valid_set's binary_error: 0.00129562
[3000]	valid_set's binary_error: 0.000971712
[4000]	valid_set's binary_error: 0.00080976
[5000]	valid_set's binary_error: 0.000755776
[6000]	valid_set's binary_error: 0.000755776


	0.9992	 = Validation score   (accuracy)
	1899.46s	 = Training   runtime
	12.07s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.998	 = Validation score   (accuracy)
	5086.74s	 = Training   runtime
	0.98s	 = Validation runtime
Fitting model: RandomForestEntr ...
	To avoid this warning, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.28 to avoid the warning)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	0.9982	 = Validation score   (accuracy)
	3776.21s	 = Training   runtime
	0.79s	 = Validation runtime
Fitting model: CatBoost ...


# Autogluon 라이브러리
* Autogluon은 자동 머신러닝(AutoML) 라이브러리로, 최소한의 코드를 통해 다양한 머신러닝 작업을 쉽게 수행할 수 있도록 돕는 도구이다. 주로 탭형 데이터(표 형식 데이터), 이미지 분류, 텍스트 분석, 객체 탐지 등의 작업에 사용된다. 사용자는 복잡한 하이퍼파라미터 튜닝이나 모델 선택을 할 필요 없이, Autogluon이 자동으로 최적의 머신러닝 모델을 찾아주고 이를 학습시켜준다.

- Autogluon은 다양한 모델을 자동으로 학습하고, 여러 하이퍼파라미터 조합을 시도해 최적의 성능을 찾는다. 사용자가 일일이 모델을 정의하거나 하이퍼파라미터를 튜닝할 필요가 없다.

train_data = data.dropna() <br>
target_column = 'is_fraud' <br><br>

model = TabularPredictor(label=target_column).fit(train_data)<br>
test_data = train_data.sample(frac=0.2, random_state=10)<br>
pred = model.predict(test_data)<br><br>

print("Train Score:", model.evaluate(test_data))<br>
print("best model: ", model.get_model_best())<br>

<br><br>
* 코드 설명 : Autogluon 라이브러리의 TabularPredictor를 사용하여 자동으로 모델을 학습하고 평가하는 과정을 나타낸 것.
--------
- TabularPredictor : Autogluon의 고수준 API로, 표 형식(탭형) 데이터에 대해 다양한 머신러닝 알고리즘을 자동으로 학습시키는 도구이다.
- label=target_column : 모델이 예측할 타겟 컬럼을 지정하는 옵션이다.
- fit(train_data) :  주어진 train_data에 대해 모델을 학습시키는 과정이다. 이때 Autogluon은 여러 가지 모델을 시도하고, 그중에서 가장 성능이 좋은 모델을 자동으로 선택해 준다.